<a href="https://colab.research.google.com/github/guipantiga/CRMAnalysis/blob/main/MBA_%26_FPGrowth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer Relationship Management (CRM) - MBA using Spark

Market basket analysis is a data mining technique used by retailers to increase sales by better understanding customer purchasing patterns.

### About

*   Data source: https://www.kaggle.com/datasets/jihyeseo/online-retail-data-set-from-uci-ml-repo



In [1]:
%%capture
!pip install pyspark

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import plotly.express as px

In [3]:
# Import
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.dataframe import DataFrame
from pyspark.ml.feature import QuantileDiscretizer, Bucketizer
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local").appName("Colab").getOrCreate()
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

# Check Spark Session Information
spark

In [4]:
rawDF = spark.read.csv('/content/data.csv', encoding="ISO-8859-1", header=True)

In [5]:
# Begin the previous treatment for Online Retail
# Filtering
# Cleaning and formating
# Renaming Columns
# Creating Columns
# Aggregating

df = (rawDF
      .filter(
          (F.col("CustomerID").isNotNull())
          & (F.col('Quantity') > 0)
          & (F.col('UnitPrice') > 0)
          & (F.col('InvoiceDate').isNotNull())
          & (F.col('Description').isNotNull())
          & (F.col('Description') != " ")
          & (F.trim(F.regexp_replace(F.col('StockCode'), '[^a-zA-Z]+', '')) == "")
          )
      .withColumn('Description', F.upper(F.trim(F.regexp_replace('Description', '[^a-zA-Z0-9]+', ' '))))
      .withColumn('transactionDate',
                  F.date_format(
                      F.to_date(
                          F.lpad(F.col('InvoiceDate').substr(1,10),10,'0'), 'MM/dd/yyyy')
                      , 'yyyy-MM-dd'
                      )
                  )
      .select(
          F.col('CustomerID').cast('bigint').alias('customerID'),
          F.col('Country').cast('string').alias('additionalSegmentation'),
          F.col('InvoiceNo').cast('string').alias('invoiceNo'),
          F.col('Description').cast('string').alias('productDescription'),
          F.col('transactionDate'),
          F.col('Quantity').cast('bigint').alias('productQuantity'),
          F.col('UnitPrice').cast('double').alias('productPrice'),
          )
      .withColumn("monetaryValue",(F.col('productPrice')*F.col('productQuantity')).cast('decimal(19,2)'))
      .groupBy('customerID','additionalSegmentation','invoiceNo','productDescription','transactionDate')
      .agg(
          F.sum('productQuantity').cast('bigint').alias('productQuantityValue'),
          F.sum('productPrice').cast('double').alias('productPriceValue'),
          F.sum('monetaryValue').cast('double').alias('monetaryValue'),
          )
      # Separate dates into Year, month and days
      .withColumn("transactionDateYear", F.date_format(F.col("transactionDate"), "Y"))
      .withColumn("transactionDateMonth", F.date_format(F.col("transactionDate"), "MM"))
      .withColumn("transactionDateDay", F.date_format(F.col("transactionDate"), "dd"))
      .filter(
          (F.col('productDescription').isNotNull())
          & (~F.col('productDescription').isin(""," "))
          )
      ).repartition(20).sortWithinPartitions(['customerID','transactionDate'])

In [16]:
# I am interested to know witch product has the most agreggated value for every customer, so what I'm doing here is creating a order based on the products's value
# In addition, I an searching for only a few products to reccomend, so for every customer I am going to use only the first 10 itens
# I am also goint to filter clientes that ordered more than 1 item on their purchases
# The question remains, what if two or more product have exactly the same value? Well then I am going to order them by alphabetical order...
# Please, bare in mind that if you come across these types of problems, find a suitable way to solve them

basketData = (df
              .orderBy('customerID','productDescription')
              .dropDuplicates(['customerID', 'productDescription'])
              .withColumn("rank_order",F.rank().over(Window.partitionBy('customerID').orderBy(F.desc('monetaryValue'))))
              .sort('customerID')
              .filter(F.col('rank_order').between(1,10))
              .orderBy(['customerID','productDescription','rank_order'])
              .dropDuplicates(subset=['customerID','monetaryValue','rank_order'])
              .withColumn('maxRankValue', (F.max('rank_order').over(Window.partitionBy('customerID'))))
              .filter(F.col('maxRankValue')>1)
              )

basketData = basketData.orderBy('productPriceValue', ascending=False).groupBy("customerID").agg(F.collect_list("productDescription")).sort('customerID')

In [18]:
from pyspark.ml.fpm import FPGrowth
#Frequent Pattern Growth – FP Growth is a method of mining frequent itemsets using support, lift, and confidence.
fpGrowth = FPGrowth(itemsCol="collect_list(productDescription)", minSupport=0.006, minConfidence=0.006)
model = fpGrowth.fit(basketData)
# Display frequent itemsets.
model.freqItemsets.show(5, False)
items = model.freqItemsets
# Display generated association rules.
model.associationRules.show(5, False)
rules = model.associationRules
# transform examines the input items against all the association rules and summarize the consequents as prediction
model.transform(basketData).show(5, False)
transformed = model.transform(basketData)

# Frequency tells us how often that combination occurs in a transaction
# Confidence tells us how if the products on the left hand side occur in a order, how like it is that the product n the right hand side are them also occuring frq(Lhs, Rhs)/frq(Lhs)
# Support tells us what percentage of all transactions that combination occurs frq(Lhs, Rhs)/N
# Lift indicates the fact about wich the probability of buying the product on the right hand side increases if the procuts on the left hand side has already been bought

+-------------------------------------+----+
|items                                |freq|
+-------------------------------------+----+
|[PACK OF SIX LED TEA LIGHTS]         |45  |
|[AREA PATROLLED METAL SIGN]          |43  |
|[CHRISTMAS LIGHTS 10 VINTAGE BAUBLES]|33  |
|[JUMBO BAG PINK VINTAGE PAISLEY]     |33  |
|[PICNIC BASKET WICKER SMALL]         |59  |
+-------------------------------------+----+
only showing top 5 rows

+----------------------------+----------------------------+-------------------+------------------+---------------------+
|antecedent                  |consequent                  |confidence         |lift              |support              |
+----------------------------+----------------------------+-------------------+------------------+---------------------+
|[HOT WATER BOTTLE KEEP CALM]|[LOVE HOT WATER BOTTLE]     |0.19424460431654678|8.634485959619402 |0.006531204644412192 |
|[HOT WATER BOTTLE KEEP CALM]|[CHOCOLATE HOT WATER BOTTLE]|0.20863309352517986|7.49990

In [22]:
# Lets get the first item that we recommend for each customer
transformedDf = (transformed
                 .withColumn("recommendedProduct", F.col('prediction').getItem(0))
                 .select('customerID','recommendedProduct')
                 )

In [26]:
# As you can see, that are a lot of cusomters that did not receive any product given their purchase habits
(transformedDf
 .withColumn('counterDummy', F.lit(1))
 .groupBy('counterDummy')
 .agg(
    F.count('customerID').alias('countClients'),
    F.sum(F.when(F.col('recommendedProduct').isNotNull(), F.lit(1)).otherwise(F.lit(0))).alias('countNotNullProducts'),
    )
 .withColumn('countNullProducts', F.col('countClients')-F.col('countNotNullProducts'))
 .drop('counterDummy')
 ).show(10, False)

+------------+--------------------+-----------------+
|countClients|countNotNullProducts|countNullProducts|
+------------+--------------------+-----------------+
|4134        |2132                |2002             |
+------------+--------------------+-----------------+



In [ ]:
# This problem can be solve in a lot of different ways,
# you can trim down the products into categories or reduce their lettering,
# you can also build much more robust models to deal with propensity
# or combine simple strategies like grouping them using RFM analysis/clustering and recommending the most frequent item